In [ ]:
#install required library 
!python -m spacy download en_core_web_lg

In [2]:
#Learn to create your first chatbot using NLTK and Keras
#import the required libraries 
import en_core_web_lg # Large SpaCy model for English language
import numpy as np
import re # regular expressions
import spacy # NLU library

from collections import defaultdict
from sklearn.svm import SVC # Support Vector Classification model

In [3]:
output_format = "OUT: {output}" 

In [4]:
#version1: Use exact matches 
#hard_coded exact questions
responses_exact = {
    "what would you like to eat tonight?": "Pasta with salmon and red pesto please!",
    "what time will you be home tonight?": "I will be home around 6 pm.",
    "what is your name?": " My name is chickpea",
    "Do you love music?": "Who does not?",
    "What music do you love?" : "melody",
    "tell me a joke" : " life is a joke",
    "default": "I don't know dear!"
}

def respond_exact(text):
    response = responses_exact.get(text.lower(), responses_exact['default'])
    return(output_format.format(input=text, output=response))

In [5]:
respond_exact("who is the president of US")

"OUT: I don't know dear!"

In [6]:
#Version2: Pattern matching 
intent_keywords = {
    'dinner_preference': ['eat', 'dinner', 'food', 'cook', 'craving'],
    'arrival_time': ['time', 'when', 'get here', 'be home'],
    'Music' :[ 'tune', 'singer', 'favorite', 'melody', 'music','song'],
    'Hobby' :[ 'do','dancing', 'swimming', 'reading book', 'Tour','what','hobby'],
    'Book' :['reading','buying', 'popular', 'sad', 'romantic']
}
# Create a dictionary of patterns
patterns = {intent: re.compile('|'.join(keys)) for intent, keys in intent_keywords.items()}
# Define a function to find the intent of a message
def get_intent_re(message):
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if pattern.search(message):
            return(intent)
    else:
        return('default')

responses_re = {
    "dinner_preference":"Pasta with salmon and red pesto please!",
    "arrival_time": "I will be home around 6 pm.",
    "Music": " yah I love music, I love melody songs. What's your favorite?",
    "Hobby": "My hobby is reading detective novel and running",
    "default":"I don't know dear, but thanks for asking!"
}

def respond_re(text):
    response = responses_re.get(get_intent_re(text))
    return(output_format.format(input=text, output=response))

In [7]:
respond_re("when do you come?")

'OUT: I will be home around 6 pm.'

In [8]:
#version 3: Machine learning 
training_sentences = [
    "What would you like to have for dinner?",
    "What do you want to eat tonight?",
    "I don't know what to cook tonight.",
    "Do you have any cravings?",
    "Can I get you something to eat?", 
    "What time will you be home?",
    "How much longer will you be?",
    "When can we expect you to get here?",
    "What's taking you so long?",
    "At what hour will you be here?"
    
]

training_intents = [
    "dinner_preference",
    "dinner_preference",
    "dinner_preference",
    "dinner_preference",
    "dinner_preference",
    "arrival_time",
    "arrival_time",
    "arrival_time",
    "arrival_time",
    "arrival_time"   
]

In [9]:
#this may take a couple of seconds
nlp = en_core_web_lg.load()

In [10]:
# Initialize the array with zeros: X
X_train = np.zeros((len(training_sentences), 
              nlp('sentences').vocab.vectors_length))

for i, sentence in enumerate(training_sentences):
    # Pass each each sentence to the nlp object to create a document
    doc = nlp(sentence)
    # Save the document's .vector attribute to the corresponding row in X
    X_train[i, :] = doc.vector

In [11]:
# Create a support vector classifier
svm_model = SVC(C=1, gamma="auto", probability=True)

# Fit the classifier using the training data
svm_model.fit(X_train, training_intents)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [12]:
def get_intent_ml(text):
    doc = nlp(text)
    return(svm_model.predict([doc.vector])[0])

In [13]:
responses_ml = {
    "dinner_preference":"Pasta with salmon and red pesto please!",
    "arrival_time": "I will be home around 6 pm.",
    "default":"I like you too!"
}

def respond_ml(text):
    response = responses_ml.get(get_intent_ml(text), responses_ml["default"])
    return(output_format.format(input=text, output=response))

In [14]:
print(respond_ml("do you wanna eat?"))


OUT: Pasta with salmon and red pesto please!
